# Sampling data for testing

In [ ]:
import pandas as pd
import requests
import os

In [ ]:
file_csv = '~/code/harlqeuinht/which_horse/raw_data/combined_flat2_csv.csv'

In [ ]:
master_df = pd.read_csv(file_csv)

In [ ]:
df = master_df.sample(n=10)

In [ ]:
df

In [ ]:
columns_to_drop = df.filter(regex='[678]').columns
columns_to_drop = columns_to_drop.drop('bet365_odds')
df = df.drop(columns=(columns_to_drop))

Create a DataFrame which stores the meeting location name and long & lat. Note this has been done on the sample dataset, not the full dataset

# Locations API

In [ ]:
def get_co_ordinates(df):
        # Create a list of all unique racecourse names
        location_names = sorted(df['meeting_name'].unique())
        # Clean racecourse names so they are reconisable by the geolocation API
        locations_df = pd.DataFrame(location_names, columns=['meeting_name'])
        locations_df['location_names_cleaned'] = locations_df['meeting_name'].replace({'BANGOR-ON-DEE':'BANGOR', 'NEWMARKET (JULY)':'NEWMARKET', ' ':'_'})
        locations_df['location_names_cleaned'] = locations_df['location_names_cleaned'].str.replace(' ', '_')

        # Iterate through the locations df, generating API endpoints for each row
        for index, location in enumerate(locations_df['location_names_cleaned']):
            base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'
            api_key = os.environ['KEY']
            params = f'address={location}+racecourse&components=country:GB&key={api_key}'
            endpoint = f'{base_url}{params}'
            # Call the geolocation API, storing the results
            results = requests.get(endpoint).json()
            # Store the returned latitude and longitude data in the respective columns
            locations_df.loc[index, 'lat'] = results['results'][0]['geometry']['location']['lat']
            locations_df.loc[index, 'lng'] = results['results'][0]['geometry']['location']['lng']
        return locations_df

locations_df = get_co_ordinates(df)

In [ ]:
def get_unique_races(df):
    unique_race_days_df = pd.DataFrame({'date': df['date'], 'meeting_name':df['meeting_name']}).drop_duplicates()
    return unique_race_days_df

unique_races_df = get_unique_races(test_df)
unique_races_df

In [ ]:
unique_races_df = pd.merge(unique_races_df, locations_df, how='left', left_on='meeting_name', right_on='meeting_name')
unique_races_df

In [ ]:
def generate_endpoint(row):
        base_url = 'https://archive-api.open-meteo.com/v1/archive?'
        latitude = row['lat']
        longitude = row['lng']
        date = row['date']
        params = '&daily=temperature_2m_mean,precipitation_sum,wind_speed_10m_max&wind_speed_unit=mph'
        return f'{base_url}&latitude={latitude}&longitude={longitude}&start_date={date}&end_date={date}&{params}'

unique_races_df['endpoint'] = unique_races_df.apply(generate_endpoint, axis=1)

In [ ]:
unique_races_df.head()

In [ ]:
def call_weather_api(row):
        response = requests.get(row['endpoint'])
        data = response.json()

        temp = data['daily']['temperature_2m_mean'][0]
        precipitation = data['daily']['precipitation_sum'][0]
        wind = data['daily']['wind_speed_10m_max'][0]
        return temp, precipitation, wind

unique_races_df[['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max']] = unique_races_df.apply(call_weather_api, axis=1, result_type='expand')

In [ ]:
unique_races_df.head()

# ALTOGETHER NOW

In [ ]:
def get_weather_final(df):
    def get_co_ordinates(df):
            location_names = sorted(df['meeting_name'].unique())
            locations_df = pd.DataFrame(location_names, columns=['meeting_name'])
            locations_df['location_names_cleaned'] = locations_df['meeting_name'].replace({'BANGOR-ON-DEE':'BANGOR', 'NEWMARKET (JULY)':'NEWMARKET', ' ':'_'})
            locations_df['location_names_cleaned'] = locations_df['location_names_cleaned'].str.replace(' ', '_')

            for index, location in enumerate(locations_df['location_names_cleaned']):
                base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'
                api_key = os.environ['KEY']
                params = f'address={location}+racecourse&components=country:GB&key={api_key}'
                endpoint = f'{base_url}{params}'
                # Call the geolocation API, storing the results
                results = requests.get(endpoint).json()
                # Store the returned latitude and longitude data in the respective columns
                locations_df.loc[index, 'lat'] = results['results'][0]['geometry']['location']['lat']
                locations_df.loc[index, 'lng'] = results['results'][0]['geometry']['location']['lng']
            return locations_df

    locations_df = get_co_ordinates(df)


    def get_unique_races(df):
        unique_race_days_df = pd.DataFrame({'date': df['date'], 'meeting_name':df['meeting_name']}).drop_duplicates()
        return unique_race_days_df

    unique_race_days_df = get_unique_races(df)
    unique_race_days_df = pd.merge(unique_race_days_df, locations_df, how='left', left_on='meeting_name', right_on='meeting_name')

    def generate_endpoint(row):
            base_url = 'https://archive-api.open-meteo.com/v1/archive?'
            latitude = row['lat']
            longitude = row['lng']
            date = row['date']
            params = '&daily=temperature_2m_mean,precipitation_sum,wind_speed_10m_max&wind_speed_unit=mph'
            return f'{base_url}&latitude={latitude}&longitude={longitude}&start_date={date}&end_date={date}&{params}'

    unique_race_days_df['endpoint'] = unique_race_days_df.apply(generate_endpoint, axis=1)

    def call_weather_api(row):
            response = requests.get(row['endpoint'])
            data = response.json()

            temp = data['daily']['temperature_2m_mean'][0]
            precipitation = data['daily']['precipitation_sum'][0]
            wind = data['daily']['wind_speed_10m_max'][0]
            return temp, precipitation, wind

    unique_race_days_df[['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max']] = unique_race_days_df.apply(call_weather_api, axis=1, result_type='expand')

    updated_df = pd.merge(df, unique_race_days_df, on=['date', 'meeting_name'], how='left')
    updated_df = updated_df.drop(columns=['endpoint', 'lat', 'lng', 'location_names_cleaned'])
    return updated_df

In [ ]:
get_weather_final(df)

In [ ]:
base_url = 'https://archive-api.open-meteo.com/v1/archive?'
longitude = '-1.597'
latitude = '52.279'
start_date = '2020-01-01'
end_date = '2020-01-07'
params = 'daily=temperature_2m_mean,precipitation_sum,rain_sum,wind_speed_10m_max,wind_direction_10m_dominant'
endpoint = f'{base_url}&latitude={latitude}&longitude={longitude}&start_date={start_date}&end_date={end_date}&{params}'
endpoint

print(requests.get(endpoint).json())


In [ ]:
endpoint

In [ ]:
latitude = locations_df['latitude']
longitude = locations_df['longitude']

In [ ]:
# This code works and returns a 200 code for current weather data
response = requests.get('http://api.weatherapi.com/v1/current.json?key=581fec608fba4a5699790722240703&dt=2020-01-01&q=London')
response

In [ ]:
print(response.json())

In [ ]:
os.environ['KEY']

# Getting the weather data

In [ ]:
import pandas as pd
import requests
import os
from get_weather_function import get_weather_data
from pipeline_cleaning import clean_data

file_csv = '../raw_data/merge_dfs.csv'
df = pd.read_csv(file_csv)
clean_df = clean_data(df.copy())

In [ ]:
clean_df[['meeting_name', 'date']].drop_duplicates()

In [74]:
print(len(df['date'].unique()))
print(len(clean_df['date'].unique()))

1064
1020


### Try again :(

In [ ]:
def get_co_ordinates(df):
            # Obtain list of unique racecourse names from the 'meeting_name' column
            location_names = sorted(df['meeting_name'].unique())
            # Create a locations dataframe and clean the names of racecourses to be recognisable by a geolocation API
            locations_df = pd.DataFrame(location_names, columns=['meeting_name'])
            locations_df['location_names_cleaned'] = locations_df['meeting_name'].replace({'BANGOR-ON-DEE':'BANGOR', 'NEWMARKET (JULY)':'NEWMARKET', ' ':'_'})
            locations_df['location_names_cleaned'] = locations_df['location_names_cleaned'].str.replace(' ', '_')

            # Iterate through each row of the locations dataframe, calling the geolocation API, returning co-ordinates for each
            for index, location in enumerate(locations_df['location_names_cleaned']):
                base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'
                api_key = os.environ['KEY']
                params = f'address={location}+racecourse&components=country:GB&key={api_key}'
                endpoint = f'{base_url}{params}'
                # Call the geolocation API, storing the results
                results = requests.get(endpoint).json()
                # Store the returned latitude and longitude data in the respective columns
                locations_df.loc[index, 'lat'] = results['results'][0]['geometry']['location']['lat']
                locations_df.loc[index, 'lng'] = results['results'][0]['geometry']['location']['lng']
            return locations_df

In [68]:
locations_df = get_co_ordinates(clean_df)


KeyboardInterrupt: 

In [73]:
locations_df

,meeting_name,location_names_cleaned,lat,lng
0,AINTREE,AINTREE,53.473476,-2.954128
1,ASCOT,ASCOT,51.412167,-0.679485
2,AYR,AYR,55.467721,-4.612545
3,BANGOR-ON-DEE,BANGOR,53.004035,-2.909421
4,BATH,BATH,51.418670,-2.407209
5,BEVERLEY,BEVERLEY,53.843136,-0.458438
6,BRIGHTON,BRIGHTON,50.829821,-0.112602
7,CARLISLE,CARLISLE,54.858994,-2.928866
8,CATTERICK,CATTERICK,54.388054,-1.649181
9,CHELMSFORD,CHELMSFORD,51.842047,0.512814


In [ ]:
locations_df_saved = locations_df.copy()

In [ ]:
clean_df.shape

In [ ]:
locations_df.head()

In [ ]:
def get_unique_races(df):
    unique_races_df = pd.DataFrame({'date': clean_df['date'], 'meeting_name':clean_df['meeting_name']}).drop_duplicates()
    return unique_races_df

In [ ]:
unique_races_df = get_unique_races(clean_df)

In [ ]:
unique_races_df.shape

In [ ]:
   # Call the unique race days function, updating the unique race days dataframe and merging with the locations dataframe
unique_races_df = pd.merge(unique_races_df, locations_df, how='left', left_on='meeting_name', right_on='meeting_name')


In [ ]:
unique_races_df.head()

In [ ]:
def generate_endpoint(row):
    base_url = 'https://archive-api.open-meteo.com/v1/archive?'
    latitude = row['lat']
    longitude = row['lng']
    date = str(row['date'])[:10]
    params = '&daily=temperature_2m_mean,precipitation_sum,wind_speed_10m_max&wind_speed_unit=mph'
    return f'{base_url}&latitude={latitude}&longitude={longitude}&start_date={date}&end_date={date}&{params}'


In [ ]:
unique_races_df['endpoint'] = unique_races_df.apply(generate_endpoint, axis=1)

In [ ]:
unique_races_df.loc[0, 'endpoint']

In [ ]:
def call_weather_api(row):
    response = requests.get(row['endpoint'])
    data = response.json()

    temp = data['daily']['temperature_2m_mean'][0]
    precipitation = data['daily']['precipitation_sum'][0]
    wind = data['daily']['wind_speed_10m_max'][0]
    return temp, precipitation, wind


In [ ]:
unique_races_df[['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max']] = unique_races_df.apply(call_weather_api, axis=1, result_type='expand')


In [ ]:
unique_races_weather_df = unique_races_df

In [ ]:
unique_races_weather_df.drop(columns=['location_names_cleaned', 'lat', 'lng','endpoint'], inplace=True)

In [ ]:
unique_races_weather_df

In [ ]:
unique_races_weather_df.to_csv('../raw_data/race_day_weather.csv')

In [ ]:
# Call the API call function on the unique race days dataframe, *NOT* on the entire dataframe
# This drastically reduces the total number of API calls as there are a significant amount of duplicates in the dataset
unique_races_df[['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max']] = unique_races_df.apply(call_weather_api, axis=1, result_type='expand')

# Merge the unique race days dataframe with the main dataframe, populating on-the-day weather for each racecourse
updated_df = pd.merge(df, unique_races_df, on=['date', 'meeting_name'], how='left')
# Drop columns created throughout this function
updated_df = updated_df.drop(columns=['endpoint', 'lat', 'lng', 'location_names_cleaned'])
updated_df